# IBM Applied Data Science Capstone Project

#### Analysing the neighborhood areas in chennai and recommending a suitable place to startup a Fast Food Restaurant

## Importing required libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## Requesting source data from wikipedia

In [113]:
source = requests.get('https://en.wikipedia.org/wiki/Areas_of_Chennai').text
soup = BeautifulSoup(source, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Areas of Chennai - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XqQFOApAEJkABDVIrigAAABG","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Areas_of_Chennai","wgTitle":"Areas of Chennai","wgCurRevisionId":953024939,"wgRevisionId":953024939,"wgArticleId":5389189,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from May 2018","Use Indian English from May 2018","All Wikipedia articles written in Indian English","Geography of Chennai","Chennai-related

## Filtering the required data using soup

In [153]:
neighborhood = []

for row in soup.find_all("table", class_="wikitable")[0].findAll("tr"):
    neighborhood.append(row.text)

### Creating a dataframe

In [168]:
chennai_df = pd.DataFrame({"Neighborhood": neighborhood})

chennai_df.head(20)

,Neighborhood
0,\nLocation\n
1,\nAdambakkam\n
2,\nAdyar\n
3,\nAlandur\n
4,\nAlapakkam\n
5,\nAlwarpet\n
6,\nAlwarthirunagar\n
7,\nAmbattur\n
8,\nAminjikarai\n
9,\nAnna Nagar\n


In [169]:
chennai_df.shape

(162, 1)

In [170]:
# removing the "\n" from the dataframe
for col in chennai_df.columns:
    chennai_df[col] = chennai_df[col].str.replace('\n', '')
chennai_df.head()

,Neighborhood
0,Location
1,Adambakkam
2,Adyar
3,Alandur
4,Alapakkam


In [175]:
#removing the unwanted row from the dataframe
df_chennai= chennai_df[chennai_df.Neighborhood != "Location"].reset_index(drop=True)
df_chennai.head(20)

,Neighborhood
0,Adambakkam
1,Adyar
2,Alandur
3,Alapakkam
4,Alwarpet
5,Alwarthirunagar
6,Ambattur
7,Aminjikarai
8,Anna Nagar
9,Annanur


## Getting the coordinates of the locations

In [181]:
# defining a function to get coordinates
!pip install geocoder

import geocoder

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [183]:
# getting the coordinates
coords = [ get_latlng(neighborhood) for neighborhood in df_chennai["Neighborhood"].tolist() ]
coords

[[12.99192000000005, 80.20603000000006],
 [13.00305000000003, 80.25193000000007],
 [13.00013000000007, 80.20060000000007],
 [13.046100000000024, 80.16499000000005],
 [13.034710000000075, 80.25416000000007],
 [13.050550000000044, 80.18397000000004],
 [13.118820000000028, 80.15442000000007],
 [13.071390000000065, 80.22256000000004],
 [13.083590000000072, 80.21020000000004],
 [13.112120000000061, 80.12895000000003],
 [13.073080000000061, 80.20952000000005],
 [13.035390000000064, 80.21220000000005],
 [13.129090000000076, 80.10361000000006],
 [13.09883000000002, 80.23238000000003],
 [12.996850000000052, 80.26691000000005],
 [13.098260000000039, 80.26833000000005],
 [13.064400000000035, 80.28065000000004],
 [13.072880000000055, 80.24567000000008],
 [13.075320000000033, 80.26991000000004],
 [12.932770000000062, 80.14387000000005],
 [13.090240000000051, 80.26557000000008],
 [13.056760000000054, 80.22746000000006],
 [12.95234000000005, 80.14411000000007],
 [13.074280000000044, 80.25920000000008

In [184]:
#storing the coordinates in the dataframe
df_chennai_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [185]:
df_chennai['Latitude'] = df_chennai_coords['Latitude']
df_chennai['Longitude'] = df_chennai_coords['Longitude']

### Creating the dataframe that contains the loaction and coordinates of neighborhood in chennai

In [186]:
df_chennai

,Neighborhood,Latitude,Longitude
0,Adambakkam,12.991920,80.206030
1,Adyar,13.003050,80.251930
2,Alandur,13.000130,80.200600
3,Alapakkam,13.046100,80.164990
4,Alwarpet,13.034710,80.254160
5,Alwarthirunagar,13.050550,80.183970
6,Ambattur,13.118820,80.154420
7,Aminjikarai,13.071390,80.222560
8,Anna Nagar,13.083590,80.210200
9,Annanur,13.112120,80.128950


In [187]:
df_chennai.shape

(161, 3)

## Creating a map of chennai with neighborhood areas located

In [189]:
# generating loaction of chennai
address = 'Chennai'

geolocator = Nominatim(user_agent="my-user")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of chennai is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of chennai is 13.0801721, 80.2838331.


In [198]:
# create map of chennai using latitude and longitude values
chennai_map = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, neighborhood in zip(df_chennai['Latitude'], df_chennai['Longitude'], df_chennai['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(chennai_map)  
    
chennai_map


## Using Foursquare API to explore neighborhood

In [200]:
CLIENT_ID = 'ICWA5XT0C2GYVL3RADGCHPYP1LX3ZJMOPPGHEXOVT22TPTRE' # your Foursquare ID
CLIENT_SECRET = '20RW2SLC2UFUJ5EUTYZ4JCOHOQE1JAWQDFAVELYQE5V3EDK2' # your Foursquare Secret
VERSION = '20200429'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ICWA5XT0C2GYVL3RADGCHPYP1LX3ZJMOPPGHEXOVT22TPTRE
CLIENT_SECRET:20RW2SLC2UFUJ5EUTYZ4JCOHOQE1JAWQDFAVELYQE5V3EDK2


In [201]:
# explore neighborhoods for 100 venues and within radius of 3000m


radius = 3000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_chennai['Latitude'], df_chennai['Longitude'], df_chennai['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [202]:
# creating a dataframe containing venues

venues_chennai=pd.DataFrame(venues)

venues_chennai.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_chennai.shape)
venues_chennai.head()

(7488, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adambakkam,12.99192,80.20603,Luxe Cinemas,12.991041,80.216962,Multiplex
1,Adambakkam,12.99192,80.20603,Phoenix Market City,12.991710,80.217297,Shopping Mall
2,Adambakkam,12.99192,80.20603,Rajdhani,12.991081,80.217003,Rajasthani Restaurant
3,Adambakkam,12.99192,80.20603,Mainland China,12.991028,80.217084,Chinese Restaurant
4,Adambakkam,12.99192,80.20603,IMAX®,12.990639,80.216310,Multiplex


### Number of venues in each neighborhoord

In [203]:
venues_chennai.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adambakkam,100,100,100,100,100,100
Adyar,63,63,63,63,63,63
Alandur,71,71,71,71,71,71
Alapakkam,43,43,43,43,43,43
Alwarpet,100,100,100,100,100,100
Alwarthirunagar,72,72,72,72,72,72
Ambattur,10,10,10,10,10,10
Aminjikarai,100,100,100,100,100,100
Anna Nagar,100,100,100,100,100,100


In [204]:
print('There are {} uniques categories.'.format(len(venues_chennai['VenueCategory'].unique())))

There are 179 uniques categories.


In [210]:
# printing out the list of categories
venues_chennai['VenueCategory'].unique()[:179]

array(['Multiplex', 'Shopping Mall', 'Rajasthani Restaurant',
       'Chinese Restaurant', 'Pizza Place', "Women's Store",
       'Sandwich Place', 'Clothing Store', 'Indian Restaurant',
       'Snack Place', 'Ice Cream Shop', 'Vegetarian / Vegan Restaurant',
       'Hotel', 'Donut Shop', 'Italian Restaurant', 'Asian Restaurant',
       'Bar', 'Restaurant', "Men's Store", 'Café', 'Department Store',
       'Coffee Shop', 'Accessories Store',
       'Molecular Gastronomy Restaurant', 'Punjabi Restaurant', 'Lounge',
       'Frozen Yogurt Shop', 'Theater', 'BBQ Joint', 'Church',
       'Breakfast Spot', 'South Indian Restaurant', 'Whisky Bar',
       'Fast Food Restaurant', 'Kerala Restaurant', 'Train Station',
       'Indie Movie Theater', 'Juice Bar', 'Grocery Store',
       'North Indian Restaurant', 'Middle Eastern Restaurant',
       'Bookstore', 'Bistro', 'Beach', 'Dessert Shop', 'Bakery', 'Spa',
       'Burger Joint', 'Performing Arts Venue', 'Salon / Barbershop',
       'Gym', 'Of

## Analyzing each neighborhood

In [208]:
# one hot encoding
chennai_onehot = pd.get_dummies(venues_chennai[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chennai_onehot['Neighborhoods'] = venues_chennai['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chennai_onehot.columns[-1]] + list(chennai_onehot.columns[:-1])
chennai_onehot = chennai_onehot[fixed_columns]

print(chennai_onehot.shape)
chennai_onehot.head()

(7488, 180)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Library,Comfort Food Restaurant,Concert Hall,Convenience Store,Coworking Space,Cricket Ground,Daycare,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Herbs & Spices Store,Historic Site,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,National Park,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Resort,Rest Area,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Surf Spot,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Adambakkam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adambakkam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adambakkam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Adambakkam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [209]:
# grouping data based on neighborhood
chennai_grouped = chennai_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(chennai_grouped.shape)
chennai_grouped

(160, 180)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Burmese Restaurant,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Library,Comfort Food Restaurant,Concert Hall,Convenience Store,Coworking Space,Cricket Ground,Daycare,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Herbs & Spices Store,Historic Site,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,National Park,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Resort,Rest Area,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Surf Spot,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Store,Warehouse Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Adambakkam,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000,0.010000,0.000000,0.020000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.030000,0.000000,0.010000,0.030000,0.00,0.030000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.030000,0.180000,0.000000,0.010000,0.000000,0.020000,0.00,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.070000,0.000000,0.00,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.00000

In [240]:
len(chennai_grouped[chennai_grouped["Fast Food Restaurant"] > 0])

100

## Creating a dataframe for fast food restaurants near chennai

In [247]:
chennai_ffr = chennai_grouped[["Neighborhoods","Fast Food Restaurant"]]

In [248]:
chennai_ffr.head()

,Neighborhoods,Fast Food Restaurant
0,Adambakkam,0.050000
1,Adyar,0.031746
2,Alandur,0.028169
3,Alapakkam,0.093023
4,Alwarpet,0.000000


### Clustering the neighborhoods

In [249]:
# set number of clusters
kclusters = 5

Chennai_clustering = chennai_ffr.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Chennai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 1, 0, 1, 0, 1, 3, 0, 1])

In [251]:
# create a dataframe that includes the cluster also the top 10 venues for each neighborhood.
Chennai_merged =chennai_ffr.copy()

# add clustering labels
Chennai_merged["Cluster Labels"] = kmeans.labels_
Chennai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Chennai_merged.head()

,Neighborhood,Fast Food Restaurant,Cluster Labels
0,Adambakkam,0.050000,3
1,Adyar,0.031746,3
2,Alandur,0.028169,1
3,Alapakkam,0.093023,0
4,Alwarpet,0.000000,1


In [252]:
# merge Chennai_merged with df_chennai to add latitude/longitude for each neighborhood
Chennai_merged = Chennai_merged.join(df_chennai.set_index("Neighborhood"), on="Neighborhood")

print(Chennai_merged.shape)
Chennai_merged.head() # check the last columns!

(161, 5)


,Neighborhood,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
0,Adambakkam,0.050000,3,12.99192,80.20603
1,Adyar,0.031746,3,13.00305,80.25193
2,Alandur,0.028169,1,13.00013,80.20060
3,Alapakkam,0.093023,0,13.04610,80.16499
4,Alwarpet,0.000000,1,13.03471,80.25416


In [253]:
# sort the results by Cluster Labels
print(Chennai_merged.shape)
Chennai_merged.sort_values(["Cluster Labels"], inplace=True)
Chennai_merged

(161, 5)


,Neighborhood,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
63,Maduravoyal,0.114286,0,13.058410,80.166360
42,Kattupakkam,0.125000,0,13.041520,80.128980
44,Keelkattalai,0.083333,0,12.956150,80.178850
144,Tirusulam,0.085714,0,12.969060,80.177120
55,Koyambedu,0.090000,0,13.066050,80.197230
62,Madipakkam,0.083333,0,12.964480,80.208700
139,Thiruneermalai,0.100000,0,12.959420,80.117180
74,Mogappair,0.090909,0,13.080530,80.161160
158,Washermanpet,0.100000,0,13.109500,80.287010
137,Thirumangalam,0.100000,0,13.082850,80.196990


## Visualizing the clusters

In [255]:
# creating the map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Chennai_merged['Latitude'], Chennai_merged['Longitude'], Chennai_merged['Neighborhood'], Chennai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

### Cluster 1

In [256]:
Chennai_merged.loc[Chennai_merged['Cluster Labels'] == 0]

,Neighborhood,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
63,Maduravoyal,0.114286,0,13.05841,80.16636
42,Kattupakkam,0.125000,0,13.04152,80.12898
44,Keelkattalai,0.083333,0,12.95615,80.17885
144,Tirusulam,0.085714,0,12.96906,80.17712
55,Koyambedu,0.090000,0,13.06605,80.19723
62,Madipakkam,0.083333,0,12.96448,80.20870
139,Thiruneermalai,0.100000,0,12.95942,80.11718
74,Mogappair,0.090909,0,13.08053,80.16116
158,Washermanpet,0.100000,0,13.10950,80.28701
137,Thirumangalam,0.100000,0,13.08285,80.19699


### Cluster 2

In [257]:
Chennai_merged.loc[Chennai_merged['Cluster Labels'] == 1]

,Neighborhood,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
133,T. Nagar,0.000000,1,13.045360,80.233900
99,Pattabiram,0.000000,1,13.123330,80.059440
98,Parry's Corner,0.000000,1,13.088678,80.290398
97,Park Town,0.020000,1,13.078670,80.279560
6,Ambattur,0.000000,1,13.118820,80.154420
26,Eranavur,0.000000,1,13.187880,80.308310
9,Annanur,0.000000,1,13.112120,80.128950
132,St.Thomas Mount,0.022472,1,13.007950,80.196080
12,Avadi,0.000000,1,13.129090,80.103610
85,Nemilichery,0.000000,1,13.122180,80.050180


### Cluster 3

In [262]:
Chennai_merged.loc[Chennai_merged['Cluster Labels'] == 2]

,Neighborhood,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
30,Gerugambakkam,0.333333,2,13.01093,80.13927


### Cluster 4

In [263]:
Chennai_merged.loc[Chennai_merged['Cluster Labels'] == 3]

,Neighborhood,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
131,Sowcarpet,0.035714,3,13.091000,80.283430
134,Tambaram,0.043478,3,12.924890,80.128180
141,Thiruvanmiyur,0.050000,3,12.986190,80.260510
154,Velachery,0.064516,3,12.979380,80.218430
149,Vadapalani,0.054945,3,13.052260,80.211200
0,Adambakkam,0.050000,3,12.991920,80.206030
118,Ramavaram,0.041667,3,12.974661,80.138105
1,Adyar,0.031746,3,13.003050,80.251930
7,Aminjikarai,0.070000,3,13.071390,80.222560
11,Ashok Nagar,0.030000,3,13.035390,80.212200


### Cluster 5

In [264]:
Chennai_merged.loc[Chennai_merged['Cluster Labels'] == 4]

,Neighborhood,Fast Food Restaurant,Cluster Labels,Latitude,Longitude
67,Mandaveli,0.142857,4,13.15576,80.23048
147,Tondiarpet,0.166667,4,13.12923,80.28955
76,Moulivakkam,0.153846,4,13.01921,80.13800
50,Korukkupet,0.200000,4,13.11680,80.27726


### Observation

####  Cluster 1 , Cluster 2 , Cluster4 : The neighborhoods in this cluster have high number of Fast Food Restaurants. 


#### Cluster 5 : The neighborhoods in this cluster have low to moderate number of Fast Food Restaurants.

#### Cluster 3 : The neighborhood in this cluster has the lowest number of Fast Food Restaurants.